In [ ]:
from google.colab import drive
import pandas as pd
import os

# 기본 경로 설정
base_input_path = '/content/drive/MyDrive/여의도_불꽃축제_데이터/시간별 인구수 통합 데이터/'
base_output_path = '/content/drive/MyDrive/여의도_불꽃축제_데이터/인구종합/'

# 출력 폴더가 없으면 생성
os.makedirs(base_output_path, exist_ok=True)

# 반복 처리 (00부터 23까지)
for hour in range(24):
    # 파일 이름 생성
    file_name = f'2024-10-05_{str(hour).zfill(2)}0000.csv'
    file_path = os.path.join(base_input_path, file_name)

    # 파일 읽기 시도
    try:
        df = pd.read_csv(file_path)

        # 필요한 열만 추출
        df = df[['DPR_X_AXIS_WGS', 'DPR_Y_AXIS_WGS', 'ARV_X_AXIS_WGS', 'ARV_Y_AXIS_WGS', 'POPL_CNT']]

        # 출발 좌표에 인구 수의 절반 추가
        dpr_population = df[['DPR_X_AXIS_WGS', 'DPR_Y_AXIS_WGS', 'POPL_CNT']].rename(
            columns={'DPR_X_AXIS_WGS': 'X_COORD', 'DPR_Y_AXIS_WGS': 'Y_COORD'}
        )
        dpr_population['POPL_CNT'] /= 2  # 인구 수 절반 분배

        # 도착 좌표에 인구 수의 절반 추가
        arv_population = df[['ARV_X_AXIS_WGS', 'ARV_Y_AXIS_WGS', 'POPL_CNT']].rename(
            columns={'ARV_X_AXIS_WGS': 'X_COORD', 'ARV_Y_AXIS_WGS': 'Y_COORD'}
        )
        arv_population['POPL_CNT'] /= 2  # 인구 수 절반 분배

        # 두 데이터프레임 합치기
        all_population = pd.concat([dpr_population, arv_population])

        # 동일 좌표별 인구 수 총합 계산
        aggregated_population = all_population.groupby(['X_COORD', 'Y_COORD'])['POPL_CNT'].sum().reset_index()

        # 결과 저장 경로 생성
        output_file_name = f'2024-10-05_{str(hour).zfill(2)}_인구종합.csv'
        output_file_path = os.path.join(base_output_path, output_file_name)

        # 결과 저장
        aggregated_population.to_csv(output_file_path, index=False, encoding='utf-8-sig')

        print(f"'{file_name}' 파일의 결과가 '{output_file_name}'로 저장되었습니다.")

    except FileNotFoundError:
        print(f"'{file_name}' 파일이 존재하지 않습니다. 건너뜁니다.")
    except Exception as e:
        print(f"'{file_name}' 처리 중 오류 발생: {e}")